In [5]:
# import necessary packages
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf
from tensorflow import keras

2024-02-20 12:35:34.991443: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Preparing Data

In [ ]:
data_dir = '/Users/mario/Desktop/ingredients'

In [ ]:
# use once we upload data to github
# import pathlib
# dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
# data_dir = tf.keras.utils.get_file('ingredients', origin=dataset_url, untar=True)
# data_dir = pathlib.Path(data_dir)

In [ ]:
# create training ds
img_height,img_width=180,180
batch_size=32
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

In [ ]:
# create validation ds
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

In [ ]:
class_names = train_ds.class_names
print(class_names)

In [ ]:
plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")

# Training Model

In [7]:
# create model instance
resnet_model = tf.keras.Sequential()

# import pre-trained model
pretrained_model= tf.keras.applications.ResNet50(include_top=False,
                   input_shape=(180,180,3),
                   pooling='avg', classes=20,
                   weights='imagenet')
for layer in pretrained_model.layers:
        layer.trainable=False

# add layer for our purpose
resnet_model.add(pretrained_model)
resnet_model.add(tf.keras.layers.Flatten())
resnet_model.add(tf.keras.layers.Dense(512, activation='relu'))
resnet_model.add(tf.keras.layers.Dense(21, activation='sigmoid'))

In [8]:
resnet_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 flatten_1 (Flatten)         (None, 2048)              0         
                                                                 
 dense_2 (Dense)             (None, 512)               1049088   
                                                                 
 dense_3 (Dense)             (None, 21)                10773     
                                                                 
Total params: 24647573 (94.02 MB)
Trainable params: 1059861 (4.04 MB)
Non-trainable params: 23587712 (89.98 MB)
_________________________________________________________________


In [9]:
resnet_model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
history = resnet_model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=10
)

# Evaluating Model

In [ ]:
# plot model accuracy
fig1 = plt.gcf()
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.axis(ymin=0, ymax=1)
plt.grid()
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend(['train', 'validation'])
plt.show()

In [ ]:
# plot model loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.grid()
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epochs')
plt.legend(['train', 'validation'])
plt.show()

# Making Predictions

In [ ]:
import cv2
image = cv2.imread(str(beef[0]))
image_resized = cv2.resize(image, (img_height,img_width))
image = np.expand_dims(image_resized,axis=0)
print(image.shape)

In [ ]:
probs = resnet_model.predict(image)
print(probs)

In [ ]:
sort_cat = np.argsort(probs[0])

In [ ]:
for cat in range(20):
  print(f"{class_names[sort_cat[cat]]}: {probs[0][sort_cat[cat]]}")